In [3]:
import os
import json
import sys
sys.path.append('../..')
import matplotlib.pyplot as plt
import numpy as np
from ipywidgets import interact, IntSlider
from visualizations.solution_visualization import prepare_data_for_gantt_chart, plot_gantt_chart, plot_robot_trajectories


In [4]:
def load_dataset(problem_dir, solution_dir):
    problems = []
    solutions = []
    
    # Load all problem instances
    for file_name in sorted(os.listdir(problem_dir)):
        with open(os.path.join(problem_dir, file_name), "r") as f:
            problems.append(json.load(f))
    
    # Load all solution files
    for file_name in sorted(os.listdir(solution_dir)):
        with open(os.path.join(solution_dir, file_name), "r") as f:
            solutions.append(json.load(f))
    
    return problems, solutions

In [5]:
# Define paths
problem_dir = "/home/jakob/thesis/datasets/simple_dataset/problem_instances"
solution_dir = "/home/jakob/thesis/datasets/simple_dataset/solutions"

# Load dataset
problems, solutions = load_dataset(problem_dir, solution_dir)

In [6]:

# Create slider
slider = IntSlider(min=0, max=len(solutions)-1, step=1, description="Instance")
interact(plot_gantt_chart("bla", *solutions[instance_idx]), instance_idx=slider)

ValueError: cannot find widget or abbreviation for argument: 'title'